In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [53]:
columns = []
data = []
test_req = requests.get("https://www.pro-football-reference.com/boxscores/201709070nwe.htm")
soup = BeautifulSoup(test_req.content, "html.parser")
scoring_table = soup.find_all('h2')[0]
test_table = scoring_table.find_next('table')
if columns == []:
    columns = [th.text.strip() for th in test_table.find_all('th')][:6]
elif columns[2:] != [th.text.strip() for th in test_table.find_all('th')]:
    print(f'issue in season {season} week {week}' )
for row in test_table.find('tbody').find_all('tr'):
    row_data = [td.text.strip() for td in row.find_all('th')] + [td.text.strip() for td in row.find_all('td')]
    data.append(row_data)

In [80]:
df = pd.DataFrame(data, columns=columns)
df = df.drop(columns=['Tm','Detail'])

df['num_seconds'] = df['Time'].str.split(":").str[0].astype(int)*60+df['Time'].str.split(":").str[1].astype(int)

index_4 = df.loc[df['Quarter']=='4',:].index[0]
index_3 = df.loc[df['Quarter']=='3',:].index[0]
index_2 = df.loc[df['Quarter']=='2',:].index[0]
index_1 = df.loc[df['Quarter']=='1',:].index[0]

df.loc[index_1:index_2-1,'num_seconds'] = df.loc[index_1:index_2-1,'num_seconds'] + 15*60*3
df.loc[index_2:index_3-1,'num_seconds'] = df.loc[index_2:index_3-1,'num_seconds'] + 15*60*2
df.loc[index_3:index_4-1,'num_seconds'] = df.loc[index_3:index_4-1,'num_seconds'] + 15*60*1

df.iloc[:,2] = df.iloc[:,2].astype(int)
df.iloc[:,3] = df.iloc[:,3].astype(int)

df['within_4'] = (abs(df.iloc[:,3] - df.iloc[:,2])<=4).astype(int)
df

/var/folders/td/dtym05z13119cjklqb5tyd040000gn/T/ipykernel_15928/3728402258.py:15: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,2] = df.iloc[:,2].astype(int)
/var/folders/td/dtym05z13119cjklqb5tyd040000gn/T/ipykernel_15928/3728402258.py:16: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,3] = df.iloc[:,3].astype(int)


,Quarter,Time,KAN,NWE,num_seconds,within_4
0,1,12:08,0,7,3428,0
1,,3:03,7,7,2883,1
2,2,14:56,7,10,2696,1
3,,7:27,7,17,2247,0
4,,0:13,14,17,1813,1
5,3,9:19,21,17,1459,1
6,,5:01,21,24,1201,1
7,,0:09,21,27,909,0
8,4,14:07,28,27,847,1
9,,5:14,35,27,314,0


In [81]:
secs_remaining = 3600
num = 0
counting = True
for i in range(len(df)):
    if counting == True:
        if df.iloc[i,:]['within_4'] == 1:
            continue
        else:
            num = num + (secs_remaining - df.iloc[i,:]['num_seconds'])
            counting = False
            secs_remaining = df.iloc[i,:]['num_seconds']
    else:
        if df.iloc[i,:]['within_4'] == 0:
            continue
        else:
            counting = True
            secs_remaining = df.iloc[i,:]['num_seconds']
if counting == True:
    num = num + secs_remaining
prop = num/3600
prop

0.6236111111111111